<h1 align="center">📘 Research Agent Analyst</h1>

<p align="center">
A fully automated AI agent that reads research PDFs, extracts and restructures
their content, builds context, summarizes them in a student-friendly way, and
evaluates the output for clarity and correctness — all using Vertex AI and
Google’s Agent Development Kit (ADK).
</p>


# 🔧 Environment Setup
Before building our research agent, we must install all required Google ADK and
Vertex AI packages. These libraries allow our notebook to deploy agents, interact
with the Reasoning Engine, and process PDFs.  
This section ensures your environment has all the dependencies needed for the
rest of the pipeline.


In [ ]:
!pip install -U google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.0/299.0 kB 25.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.5.0
    Uninstalling tenacity-8.5.0:
      Successfully uninstalled tenacity-8.5.0
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.19.0
    Uninstalling google-cloud-storage-2.19.0:
      Successfully uninstalled google-cloud-storage-2.19.0
  Attempting uninstall: google-adk
    Found existing installation: google-adk 1.17.0
    Uninstalling google-adk-1.17.0:
      Successfully uninstalled google-adk-1.17.0


In [ ]:
!pip install google-cloud-aiplatform[adk,agent_engines] google-adk


INFO: pip is looking at multiple versions of opentelemetry-instrumentation to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 2.0.1
    Uninstalling wrapt-2.0.1:
      Successfully uninstalled wrapt-2.0.1


# 📦 Importing Required Libraries
Here we load all the Python libraries used throughout the project.This includes Vertex AI, ADK components, PDF readers, and utility modules.  


In [ ]:
import os
import random
import time
import vertexai
from vertexai import agent_engines

print("✅ Imports completed successfully")

✅ Imports completed successfully


# 🔐 Configuring API Keys and Project Settings
In this section, we connect our notebook to the correct Google Cloud project.  
We set environment variables for:
- Project ID  
- API keys  
- Region  
These values tell Vertex AI and ADK where to deploy and run our agent.


In [ ]:
from google.colab import userdata
import os
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

In [ ]:
## Set your PROJECT_ID
PROJECT_ID = "my-research-agent-478404"  # TODO: Replace with your project ID
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

if PROJECT_ID == "your-project-id" or not PROJECT_ID:
    raise ValueError("⚠️ Please replace 'your-project-id' with your actual Google Cloud Project ID.")

print(f"✅ Project ID set to: {PROJECT_ID}")

✅ Project ID set to: my-research-agent-478404


# 📁 Preparing Project Structure
We create a dedicated folder (`research_agent/`) to store all files for the
research agent — such as Python scripts, requirements, environment configs, and
agent engine files.  



In [ ]:
!mkdir -p research_agent


In [ ]:
%%writefile research_agent/requirements.txt
google-adk
google-cloud-aiplatform[adk,agent_engines]>=1.111
opentelemetry-instrumentation-google-genai
pypdf
requests
google-genai



Writing research_agent/requirements.txt


In [ ]:
%%writefile research_agent/.env

# https://cloud.google.com/vertex-ai/generative-ai/docs/learn/locations#global-endpoint
GOOGLE_CLOUD_PROJECT="my-research-agent-478404"
GOOGLE_CLOUD_LOCATION="us-east4"
GOOGLE_GENAI_USE_VERTEXAI=1

Writing research_agent/.env


In [ ]:
%%writefile research_agent/.agent_engine_config.json
{
  "min_instances": 0,
  "max_instances": 1,
  "resource_limits": {
    "cpu": "1",
    "memory": "2Gi"
  }
}


Writing research_agent/.agent_engine_config.json


In [ ]:
import vertexai
import os

PROJECT_ID = "my-research-agent-478404"
REGION = "us-east4"
BUCKET = "gs://my-research-agent-staging"

# MUST BE SET BEFORE vertexai.init()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/my-research-agent-478404-594c8dc978b0.json"

vertexai.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=BUCKET,
)

print("Vertex AI initialized with service account JSON")


Vertex AI initialized with service account JSON


In [ ]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/my-research-agent-478404-594c8dc978b0.json"

print("Credentials set.")


Credentials set.


# 🤖 Building the Research Agent
In this section, we define the entire agent pipeline:
- PDF reader tool  
- Document reader agent  
- Context builder agent  
- Summarizer agent  
- Evaluator agent  
- Root orchestrator agent  
This is the heart of the project — it automates the full research-paper workflow.


In [ ]:
%%writefile research_agent/agent.py
import os
import tempfile

import vertexai
from google.genai import types

from google.adk.agents import Agent, LlmAgent
from google.adk.tools import AgentTool
from google.adk.models.google_llm import Gemini

from pypdf import PdfReader
from google.cloud import storage

# -------------------------------------------------------------------
# Vertex AI init
# -------------------------------------------------------------------
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]
LOCATION = os.environ["GOOGLE_CLOUD_LOCATION"]
BUCKET = "gs://my-research-agent-staging"

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=BUCKET,
)

# -------------------------------------------------------------------
# PDF Reader – plain Python tool function (NO AgentTool subclass)
# -------------------------------------------------------------------
def pdf_reader_tool(file_path: str) -> dict:
    """
    Reads a PDF either from local path or GCS (gs://bucket/path.pdf)
    and returns the extracted text.

    Args:
        file_path: Local path or GCS URI to the PDF.

    Returns:
        dict: {"status": "success", "text": "<full extracted text>"} or
              {"status": "error", "error": "<message>"} on failure.
    """
    try:
        # Create a temporary local file to store the PDF
        temp_pdf = tempfile.NamedTemporaryFile(
            delete=False, suffix=".pdf"
        ).name

        # If path is GCS, download first
        if file_path.startswith("gs://"):
            storage_client = storage.Client()
            path_no_scheme = file_path.replace("gs://", "", 1)
            bucket_name = path_no_scheme.split("/")[0]
            blob_name = "/".join(path_no_scheme.split("/")[1:])

            bucket = storage_client.bucket(bucket_name)
            blob = bucket.blob(blob_name)
            blob.download_to_filename(temp_pdf)
            local_path = temp_pdf
        else:
            # Treat as local file path
            local_path = file_path

        # Extract text from PDF
        reader = PdfReader(local_path)
        text = "\n".join(page.extract_text() or "" for page in reader.pages)

        return {"status": "success", "text": text}

    except Exception as e:
        return {"status": "error", "error": str(e)}


# -------------------------------------------------------------------
# Retry config for all LLM agents
# -------------------------------------------------------------------
retry = types.HttpRetryOptions(
    attempts=5,
    exp_base=2,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

# -------------------------------------------------------------------
# Agent 1 – Document Reader
# -------------------------------------------------------------------
document_reader = LlmAgent(
    name="document_reader",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry),
    instruction="""
You receive the FULL TEXT of a research paper.

Your job is to transform it into CLEAN, STRUCTURED MARKDOWN:

- Title
- Authors (if available)
- Abstract
- Sections with headings
- Important equations (LaTeX if possible)
- Tables (in text form)

RULES:
- Do NOT summarize.
- Do NOT add your own commentary.
- Just restructure and clean the content.
""",
)

# -------------------------------------------------------------------
# Agent 2 – Context Builder
# -------------------------------------------------------------------
context_builder = LlmAgent(
    name="context_builder",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry),
    instruction="""
You receive structured markdown for a research paper.

Compress it into 400–600 words while preserving:
- Motivation / problem
- Methods / approach
- Datasets / inputs
- Results
- Conclusions

RULES:
- Do NOT invent details.
- Keep wording clear and dense.
""",
)

# -------------------------------------------------------------------
# Agent 3 – Summarizer
# -------------------------------------------------------------------
summarizer = LlmAgent(
    name="summarizer",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry),
    instruction="""
You receive a compressed 400–600 word context about a research paper.

Produce a STUDENT-FRIENDLY summary with these sections:

1. Title
2. Problem
3. Background / Motivation
4. Method / Model
5. Dataset / Inputs
6. Results (include important numbers if present)
7. Strengths
8. Limitations
9. Key Takeaways (5–10 bullet points)
10. Overall Conclusion

RULES:
- Length: about 300–600 words.
- Use simple, clear English suitable for a CS/AI student.
""",
)

# -------------------------------------------------------------------
# Agent 4 – Evaluator
# -------------------------------------------------------------------
evaluator = LlmAgent(
    name="evaluator",
    model=Gemini(model="gemini-2.0-flash", retry_options=retry),
    instruction="""
You receive:
- The compressed context of a research paper.
- A candidate summary written for students.

Evaluate ONLY on:
- Technical correctness
- Clarity
- Completeness (are key ideas covered?)

Output:
- SHORT bullet-point feedback listing issues or confirming quality.

Do NOT rewrite the summary.
""",
)

# -------------------------------------------------------------------
# Wrap the LLM agents as AgentTools
# -------------------------------------------------------------------
document_reader_tool = AgentTool(agent=document_reader)
context_builder_tool = AgentTool(agent=context_builder)
summarizer_tool = AgentTool(agent=summarizer)
evaluator_tool = AgentTool(agent=evaluator)

# -------------------------------------------------------------------
# ROOT AGENT – Orchestrator
# -------------------------------------------------------------------
root_agent = Agent(
    name="research_analyst",
    model="gemini-2.0-flash",
    description=(
        "Reads research PDFs (from GCS or local path), extracts text, and "
        "produces detailed, student-friendly summaries."
    ),
    instruction="""
You are the RESEARCH ANALYST AGENT.

Your pipeline MUST follow these steps in order:

1️⃣ Call `pdf_reader_tool(file_path=...)`
    - Input is a GCS URI (gs://...) or local path to a PDF.
    - If the tool returns status != "success", explain the error.

2️⃣ Call `document_reader` on the extracted text to produce CLEAN MARKDOWN.

3️⃣ Call `context_builder` on that markdown to get a 400–600 word context.

4️⃣ Call `summarizer` on the context to produce a student-friendly summary.

5️⃣ Call `evaluator` with BOTH the context and the summary.
    - Use the evaluator's feedback to fix any obvious mistakes.

6️⃣ Return ONLY the final improved summary to the user.
    - Do NOT include raw PDF text.
    - Do NOT include intermediate tool outputs.
    - Do NOT include evaluator comments in the final answer.

FINAL SUMMARY RULES:
- 300–600 words.
- Clear, organized, and easy to understand for a CS/AI student.
""",
    tools=[
        pdf_reader_tool,          # plain Python function tool
        document_reader_tool,     # LLM tools
        context_builder_tool,
        summarizer_tool,
        evaluator_tool,
    ],
)


Writing research_agent/agent.py


# 🚀 Deploying Agent Engine to Vertex AI
We package all files, build the agent engine, and deploy it to Google Cloud.  
Once deployed, the agent becomes available as a cloud service you can query
remotely.


In [ ]:
!adk deploy agent_engine \
 --project=$PROJECT_ID \
 --region=us-east4 \
 research_agent \
 --agent_engine_config_file=research_agent/.agent_engine_config.json


Staging all files in: /content/research_agent_tmp20251118_041236
Copying agent source code...
Copying agent source code complete.
Resolving files and dependencies...
Reading agent engine config from research_agent/.agent_engine_config.json
Reading environment variables from /content/research_agent/.env
Ignoring GOOGLE_CLOUD_PROJECT in .env as `--project` was explicitly passed and takes precedence
Ignoring GOOGLE_CLOUD_LOCATION in .env as `--region` was explicitly passed and takes precedence
Initializing Vertex AI...
Vertex AI initialized.
Created research_agent_tmp20251118_041236/agent_engine_app.py
Files and dependencies resolved
Deploying to agent engine...
✅ Created agent engine: projects/610233153602/locations/us-east4/reasoningEngines/7301707186479038464
Cleaning up the temp folder: research_agent_tmp20251118_041236


# 🔗 Connecting to Deployed Agent
After deployment, we fetch the agent from Vertex AI.  
This allows us to send queries, upload PDFs, and run the agent pipeline in the cloud.


In [ ]:
deployed_region = "us-east4"
# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=deployed_region)

# Get the most recently deployed agent
agents_list = list(agent_engines.list())
if agents_list:
    remote_agent = agents_list[0]  # Get the first (most recent) agent
    client = agent_engines
    print(f"✅ Connected to deployed agent: {remote_agent.resource_name}")
else:
    print("❌ No agents found. Please deploy first.")

✅ Connected to deployed agent: projects/610233153602/locations/us-east4/reasoningEngines/7301707186479038464


# 📤 Uploading PDF to Google Cloud Storage
We upload a PDF document to our GCS bucket.  
The agent will read this file directly during processing.


In [ ]:
from google.genai import types

pdf_part = types.Part.from_uri(
    file_uri="gs://my-research-agent-staging/uploads/ase10.pdf",
    mime_type="application/pdf",
)

async for event in remote_agent.async_stream_query(
    user_id="u1",
    message="Process this PDF and return final research summary.",
    attachments=[pdf_part],
):
    print(event)


In [ ]:
!gcloud auth activate-service-account --key-file="/content/my-research-agent-478404-594c8dc978b0.json"
!gcloud config set project my-research-agent-478404


In [ ]:
!gsutil cp "/content/ase_10.pdf" gs://my-research-agent-staging/uploads/ase10.pdf


Copying file:///content/ase_10.pdf [Content-Type=application/pdf]...
/ [1 files][535.5 KiB/535.5 KiB]                                                
Operation completed over 1 objects/535.5 KiB.                                    


#Sample Test

In [ ]:
async for event in remote_agent.async_stream_query(
    user_id="u1",
    message="Hello, what can you do?"
):
    print(event)


{'model_version': 'gemini-2.0-flash', 'content': {'parts': [{'text': 'I can read research PDFs, extract the text, create a context, summarize the content in a student-friendly manner, and evaluate the summary for improvements. What PDF would you like me to process? Please provide the file path.\n'}], 'role': 'model'}, 'finish_reason': 'STOP', 'usage_metadata': {'candidates_token_count': 47, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 47}], 'prompt_token_count': 434, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 434}], 'total_token_count': 481, 'traffic_type': 'ON_DEMAND'}, 'avg_logprobs': -0.09033814896928503, 'invocation_id': 'e-3df69f02-7b51-4987-9999-67b14bde1b68', 'author': 'research_analyst', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}, 'requested_tool_confirmations': {}}, 'id': '147950fd-b574-4eb1-a311-797fde81a553', 'timestamp': 1763439567.787122}


# 🧠 Running the Research Agent End-to-End
Finally, we send a query to the deployed agent along with the PDF.  
The agent:
1. Reads the PDF  
2. Cleans the text  
3. Builds a context  
4. Summarizes  
5. Evaluates & improves  
6. Returns the final summary  

This is where everything comes together and you see the full pipeline in action.


In [ ]:
async for event in remote_agent.async_stream_query(
    user_id="u1",
    message=(
        "Process this PDF and give me the final summary. "
        "file_path=gs://my-research-agent-staging/uploads/ase10.pdf"
    ),
):
    print(event)


{'model_version': 'gemini-2.0-flash', 'content': {'parts': [{'text': "Okay, I will process the PDF from the given GCS path and provide you with a student-friendly summary. Here's the plan:\n\n1.  **Read the PDF:** Use `pdf_reader_tool` to extract text from the PDF.\n2.  **Clean the text:** Use `document_reader` to clean the extracted text and convert it to markdown.\n3.  **Build context:** Use `context_builder` to create a detailed context from the markdown.\n4.  **Summarize:** Use `summarizer` to generate an initial summary of the context.\n5.  **Evaluate:** Use `evaluator` to evaluate the summary and context.\n6.  **Improve:** Based on the evaluation, refine the summary.\n7.  **Return:** Provide you with the final, improved summary.\n\nLet's start by reading the PDF.\n\n"}, {'function_call': {'id': 'adk-eba62ace-23ad-4d90-8476-f20bda6cdbbf', 'args': {'file_path': 'gs://my-research-agent-staging/uploads/ase10.pdf'}, 'name': 'pdf_reader_tool'}}], 'role': 'model'}, 'finish_reason': 'STO

# Deleting Resources Created

In [ ]:
from vertexai import agent_engines
import vertexai

vertexai.init(project="my-research-agent-478404", location="us-west1") #east4 also delete

agents = list(agent_engines.list())
for a in agents:
    print("Deleting:", a.resource_name)
    agent_engines.delete(a.resource_name, force=True)
